# Basic scraping and demo

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
#from retrying import retry

In [2]:
#@retry(stop_max_attempt_number=10, wait_random_min=1000, wait_random_max=2000)
def get_vcb_exchange_rate(source_url):
    # print('Connecting to VCB')
    r = requests.get(source_url)
    soup = BeautifulSoup(r.content, 'xml')
    raw_datetime = soup.find('DateTime').text
    update_time = datetime.datetime.strptime(raw_datetime, '%m/%d/%Y %I:%M:%S %p')
    Exrate = soup.findAll('Exrate')
    Data = []
    for i in Exrate:
        CurrencyCode = i['CurrencyCode'].strip()
        Buy = i['Buy'].strip()
        Transfer = i['Transfer'].strip()
        Sell = i['Sell'].strip()
        Data.append([CurrencyCode, Buy, Transfer, Sell, update_time])
    df = pd.DataFrame(Data, columns=['CurrencyCode','Buy','Transfer','Sell','update_time'])
    return df


In [3]:
source_url = 'http://www.vietcombank.com.vn/ExchangeRates/ExrateXML.aspx'
vcb_rate = get_vcb_exchange_rate(source_url)

In [4]:
vcb_rate

,CurrencyCode,Buy,Transfer,Sell,update_time
0,AUD,15611.28,15768.97,16101.43,2020-01-10 19:57:15
1,CAD,17374.15,17549.65,17919.65,2020-01-10 19:57:15
2,CHF,23310.02,23545.47,24041.88,2020-01-10 19:57:15
3,DKK,0,3379.43,3506.35,2020-01-10 19:57:15
4,EUR,25319.61,25575.36,26343.5,2020-01-10 19:57:15
5,GBP,29842.27,30143.71,30472.95,2020-01-10 19:57:15
6,HKD,2932.29,2961.91,3006.26,2020-01-10 19:57:15
7,INR,0,325.52,338.3,2020-01-10 19:57:15
8,JPY,204.35,206.41,212.39,2020-01-10 19:57:15
9,KRW,18.27,19.23,21.07,2020-01-10 19:57:15


# Auto email

## Email sender
Allow less secure apps must be on

Follow this link to change your setting https://myaccount.google.com/lesssecureapps?pli=1

In [ ]:
import pandas
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
def vcb_email():
    sender_id = ''  
    sender_password = ''
    sender = ''
    recipients = ['']
    subject = 'Vcb exchange rate report at {}'.format(datetime.datetime.now())

    # Create the body of the message (a plain-text and an HTML version).
    text = 'Vcb exchange rate report at {}'.format(datetime.datetime.now())
    html = vcb_rate.to_html()

    msg = MIMEMultipart("alternative", None, [MIMEText(text), MIMEText(html,'html')])
    msg['Subject'] = subject
    msg['From'] = self.sender
    msg['To'] = ", ".join(recipients)   
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.ehlo()
    server.login(sender_id, sender_password)
    server.sendmail(sender, recipients, msg.as_string()) 
    server.quit()
    print ('send mail ok')


## Scheduler

In [ ]:
import datetime
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.triggers.interval import IntervalTrigger
import csv

In [ ]:
#create a log file
def scheduler_logging(content):
    log_folder = 'C:/Users/hoang.to/Dropbox/SchedLog/'
    filename = 'masterlog_{}.csv'.format(datetime.datetime.now().strftime('%d%b%Y'))
    ffilename = log_folder + filename
    with open(ffilename, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(content)
#def job
def job_vcb_email():
    print('job_vcb_email start at {}'.format(datetime.datetime.now()))
    scheduler_logging(['job_vcb_email', 'Start', '{}'.format(datetime.datetime.now())])
    try:
        pass #run some code here
    
        scheduler_logging(
            ['job_vcb_email', 'Success', '{}'.format(datetime.datetime.now()),
             '{} record updated'.format(updated)])
    except:
        print('job_0 error at {}'.format(datetime.datetime.now()))
        scheduler_logging(
            ['job_vcb_email', 'Error', '{}'.format(datetime.datetime.now()),
             '{} record updated'.format(0)])
        
#create trigger
trigger_sched = BlockingScheduler(max_instances='4')
vcb_trigger = CronTrigger(day_of_week='mon-fri', hour='7-19')



job_vcb_email = trigger_sched.add_job(job_vcb_email, trigger=vcb_trigger)
                        # , kwargs={'sender_id': sender_id, 'sender_password': sender_password}) used to send var to func

scheduler_logging(['Scheduler', 'Start', '{}'.format(datetime.datetime.now())])
print('Scheduler start')
trigger_sched.start()


# Auto report

## OpenPyxl
## Pandas to excel

In [5]:
vcb_rate

,CurrencyCode,Buy,Transfer,Sell,update_time
0,AUD,15611.28,15768.97,16101.43,2020-01-10 19:57:15
1,CAD,17374.15,17549.65,17919.65,2020-01-10 19:57:15
2,CHF,23310.02,23545.47,24041.88,2020-01-10 19:57:15
3,DKK,0,3379.43,3506.35,2020-01-10 19:57:15
4,EUR,25319.61,25575.36,26343.5,2020-01-10 19:57:15
5,GBP,29842.27,30143.71,30472.95,2020-01-10 19:57:15
6,HKD,2932.29,2961.91,3006.26,2020-01-10 19:57:15
7,INR,0,325.52,338.3,2020-01-10 19:57:15
8,JPY,204.35,206.41,212.39,2020-01-10 19:57:15
9,KRW,18.27,19.23,21.07,2020-01-10 19:57:15


In [10]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, Color
from openpyxl.styles import colors


title = NamedStyle(name="Style1")
title.font = Font(size=11, color=colors.WHITE)
title.fill = PatternFill(fill_type='solid', start_color='FF1F497D', end_color='FF1F497D')

content = NamedStyle(name="content")
content.font = Font(size=8)

# load template
fn = 'vcb_rate_example.xlsx'
wb = openpyxl.load_workbook(fn)
ws = wb['template_sheet']

sheetname = 'template_sheet'
ws = wb[sheetname]

for r in dataframe_to_rows(vcb_rate, index=False, header=True):
    ws.append(r)

print('Writing ' + sheetname + ' Done')

for cell in ws[10]:
    cell.style = title

filename = 'Vcb_exchange_rate{}.xlsx'.format(datetime.date.today().strftime('%d%b%Y'))
wb.save(filename)

Writing template_sheet Done


## 4. Connect to SQL Server

In [ ]:
import pyodbc 
conn = pyodbc.connect(
    "Driver={SQL Server Native Client 11.0};"
    "Server= 103.137.4.34,9900;"
    "Database=AdventureWorksDW2012;"
    "UID=datapot_student;"
    "PWD=datapotvn"
)

cursor = conn.cursor()
cursor.execute('SELECT * FROM dbo.[DimDepartmentGroup]')

for row in cursor:
    print(row)

In [ ]:
df_company_list = pd.read_sql('SELECT * FROM dbo.[DimDepartmentGroup]', conn)